In [1]:
import pandas as pd

In [2]:
# TODO: 各自のサブミット用データのパスに書き換える
submission = pd.read_csv("./submit/submission.csv")

In [3]:
train = pd.read_csv("./data/train_data.csv")
test = pd.read_csv("./data/test_data.csv")

In [4]:
submission = (
    submission.
    merge(test[["id", "host_id", "name", "room_type", "neighbourhood"]], how = "left", on = "id")
)

# 高額施設 "Uhome ..." を90万円に置き換える
# "Uhome ..." だけでなく、同じホストが運営する他施設も同様に置き換える
submission = (
    submission.
    assign(y = lambda df:
        df["y"].mask(
            df["host_id"].isin(df.loc[lambda df: df["name"].str.lower().str.contains("uhome"), "host_id"]),
            900000
        )
    )
)

# 日割料金ではなく誤って月額料金を設定してしまっていると思われる施設 "★Monthly Rental ..." の
# 一軒家を20万円に、一軒家以外を10万円に置き換える
# "★Monthly Rental ..." だけでなく、同じホストが運営する他施設も同様に置き換える
submission = (
    submission.
    assign(y = lambda df:
        df["y"].mask(
            df["host_id"].isin(df.loc[lambda df: df["name"].str.contains("★Monthly Rental"), "host_id"]) & df["name"].str.lower().str.contains("house"),
            200000
        )
    ).
    assign(y = lambda df:
        df["y"].mask(
            df["host_id"].isin(df.loc[lambda df: df["name"].str.contains("★Monthly Rental"), "host_id"]) & ~df["name"].str.lower().str.contains("house"),
            100000
        )
    )
)

# シェアルームタイプの施設の予測結果が学習データの分布より上振れしているため、
# 学習データの価格の中央値に置き換える
submission = (
    submission.
    # 所在地別の中央値に置き換える
    merge(train.groupby(["room_type", "neighbourhood"])["y"].median().rename("y_median"), how = "left", on = ["room_type", "neighbourhood"]).
    assign(y = lambda df: df["y"].mask(df["room_type"] == "Shared room", df["y_median"])).
    drop(columns = ["y_median"]).
    # 学習データにシェアルームタイプの施設がない所在地については、全体の中央値に置き換える
    merge(train.groupby(["room_type"])["y"].median().rename("y_median"), how = "left", on = ["room_type"]).
    assign(y = lambda df: df["y"].mask(df["y"].isna(), df["y_median"])).
    drop(columns = ["y_median"])
)

# 施設名称に "Simple" が含まれる施設の予測結果が学習データの分布より上振れしているため、
# 学習データの価格の中央値に置き換える
submission = (
    submission.
    merge(train.loc[lambda df: df["name"].str.contains("Simple"), :].groupby(["room_type"])["y"].median().rename("y_median"), how = "left", on = ["room_type"]).
    assign(y = lambda df: df["y"].mask(df["name"].str.contains("Simple"), df["y_median"])).
    drop(columns = ["y_median"])
)

submission = submission[["id", "y"]].copy()

In [5]:
# TODO: 各自の出力先に書き換える
submission.to_csv("./submit/submission.csv", index = False, header = True)